In [69]:
import pandas as pd
import numpy as np
import glob, os
import xlrd
import json
import  asposecells     


In [90]:
pd.set_option('display.max_rows',50)
pd.set_option('display.max_columns',50)


In [54]:
result_files = glob.glob(os.path.join(f'{os.getcwd()}\\Report\\', 'V-EP6*_*.xls'))
result_files.sort()
result_files

['d:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230321_122611_Fail_1.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230321_122642_Fail_2.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230321_122740_Fail_3.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230321_123338_Fail_4.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120154_325.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120227_326.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120248_Fail_35.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120307_Fail_36.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120332_327.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120354_328.xls',
 'd:\\MHC\\2023\\개인자료\\kmong\\조금예민한 독수리\\Report\\V-EP6XA197AA_20230324_120519_329.xls',
 'd:\\MHC\\2

In [55]:
len(result_files)

7053

In [56]:
valid_485 = ["AC 220V 전원 인가","AC 220V 전원 검사", "DC12V 전원 검사", "DC 5V 전원 검사", 
             "TEST 시작", "COM0 MODE 485", "H/W MODEL","F/W VERSION","UNIT TYPE", "EEPROM CHECK", "COM0 TEST", "COM1 TEST", 
             "SENSOR0 (TRA)", "SENSOR1 (TDF)", "RELAY0 COMP ON", "RELAY0 COMP OFF", "RELAY1 ALARM ON", "RELAY1 ALARM OFF"
             "RELAY2 DEF ON", "RELAY2 DEF OFF", "RELAY3 FAN ON", 'RELAY3 FAN OFF', "DISP MOSI HIGH", "DISP MOSI LOW", "DISP MISO HIGH",
             "DISP MISO LOW", "DISP CLK HIGH", "DISP CLK LOW", "DISP STB HIGH", "DISP STB LOW", "AC 220V 전원 OFF"]

valid_ttl = ["AC 220V 전원 인가","AC 220V 전원 검사", "DC12V 전원 검사", "DC 5V 전원 검사", 
           "TEST 시작", "COM0 MODE TTL", "H/W MODEL","F/W VERSION","UNIT TYPE", "EEPROM CHECK", "COM0 TEST", "COM1 TEST", 
           "SENSOR0 (TRA)", "SENSOR1 (TDF)", "RELAY0 COMP ON", "RELAY0 COMP OFF", "RELAY2 DEF ON", "RELAY2 DEF OFF",
           "RELAY3 FAN ON", 'RELAY3 FAN OFF', "DISP MOSI HIGH", "DISP MOSI LOW", "DISP MISO HIGH",
           "DISP MISO LOW", "DISP CLK HIGH", "DISP CLK LOW", "DISP STB HIGH", "DISP STB LOW", "COM0 TXEN HIGH", "COM0 TXEN LOW" "AC 220V 전원 OFF"]



In [63]:
import win32com.client

type1, type2 = 0, 0

for f in result_files:
    info = f.split('\\')[-1].split('_')
    date, qty = info[1], info[2]
    excel = win32com.client.Dispatch("Excel.Application")
    workbook = excel.Workbooks.Open(f)

    # 시트 가져오기
    sheet = workbook.Sheets[1]

    # 데이터 가져오기
    data = sheet.Range("A1:G40").Value

    df = pd.DataFrame(data)

    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0]).reset_index(drop=True)

    com_type = df[(df['명령어']==8) |(df['명령어']==9)]['검사항목'].values[0]

    if 'Fail(실패)' in list(df['OK 상태 확인'].values):
        fail_reason = df[df['OK 상태 확인']=='Fail(실패)']['불량내용'].values[0]
    else:
        fail_reason = 'OK'

    if '485' in com_type:
    
        df_485 = df[df['검사항목'].isin(valid_485)]
        if type1 ==0:
            limit_485 = dict()
            for v in df_485[['검사항목', '측정 DATA1 상한', '측정 DATA1 하한']].values:
                ins, ucl, lcl = v
                limit_485[ins]= dict()
                limit_485[ins]['ucl'] = ucl
                limit_485[ins]['lcl'] = ucl

        df_485 = pd.DataFrame(df_485[['측정 DATA1']].values.T, columns=df_485.검사항목.values)
        df_485['date'] = date
        df_485['qty'] = qty
        new_cols = ['date', 'qty'] + list(df_485.columns)
        df_485 = df_485[new_cols]
        if fail_reason == '0:실패, 1:성공':
            df_485['COM0 MODE 485'] = 0
        else:
            df_485['COM0 MODE 485'] = 1
        df_485['Status'] = fail_reason
        df_type1 = df_485 if type1 ==0 else pd.concat([df_type1, df_485], axis=0)
        type1 +=1

    elif 'TTL' in com_type:
        df_ttl = df[df['검사항목'].isin(valid_ttl)]
        if type2 ==0:
            limit_ttl = dict()
            for v in df_ttl[['검사항목', '측정 DATA1 상한', '측정 DATA1 하한']].values:
                ins, ucl, lcl = v
                limit_ttl[ins]= dict()
                limit_ttl[ins]['ucl'] = ucl
                limit_ttl[ins]['lcl'] = ucl
        df_ttl = pd.DataFrame(df_ttl[['측정 DATA1']].values.T, columns=df_ttl.검사항목.values)
        df_ttl['date'] = date
        df_ttl['qty'] = qty
        new_cols = ['date', 'qty'] + list(df_ttl.columns)
        df_ttl = df_ttl[new_cols]
        if fail_reason == '0:실패, 1:성공':
            df_ttl['COM0 MODE TTL'] = 0
        else:
            df_485['COM0 MODE TTL'] = 1
        df_ttl['Status'] = fail_reason
        df_type2 = df_ttl if type2 ==0 else pd.concat([df_type2, df_ttl], axis=0)
        type2 +=1

    workbook.Close()
    excel.Quit()


In [66]:
df_type1.to_csv('com_485.csv', index=False)

In [67]:
df_type2.to_csv('com_ttl.csv', index=False)

In [82]:

with open("limit_ttl.json", "w",encoding='utf-8') as outfile:
    json.dump(limit_ttl, outfile, indent='\t',ensure_ascii=False)

In [83]:

with open("limit_485.json", "w",encoding='utf-8') as outfile:
    json.dump(limit_485, outfile,indent='\t',ensure_ascii=False)

In [87]:
with open("limit_ttl.json", 'r', encoding='utf-8') as file:
    data = json.load(file)
    print(type(data))
    print(data)


<class 'dict'>
{'AC 220V 전원 인가': {'ucl': 0.0, 'lcl': 0.0}, 'AC 220V 전원 검사': {'ucl': 230.0, 'lcl': 230.0}, 'DC12V 전원 검사': {'ucl': 12.36, 'lcl': 12.36}, 'DC 5V 전원 검사': {'ucl': 5.15, 'lcl': 5.15}, 'TEST 시작': {'ucl': 1.0, 'lcl': 1.0}, 'COM0 MODE TTL': {'ucl': 1.0, 'lcl': 1.0}, 'H/W MODEL': {'ucl': 16.0, 'lcl': 16.0}, 'F/W VERSION': {'ucl': 0.46, 'lcl': 0.46}, 'UNIT TYPE': {'ucl': 1.0, 'lcl': 1.0}, 'EEPROM CHECK': {'ucl': 1.0, 'lcl': 1.0}, 'COM0 TEST': {'ucl': 1.0, 'lcl': 1.0}, 'COM1 TEST': {'ucl': 1.0, 'lcl': 1.0}, 'SENSOR0 (TRA)': {'ucl': 3.3, 'lcl': 3.3}, 'SENSOR1 (TDF)': {'ucl': 3.5, 'lcl': 3.5}, 'RELAY0 COMP ON': {'ucl': 1.0, 'lcl': 1.0}, 'RELAY0 COMP OFF': {'ucl': 0.0, 'lcl': 0.0}, 'RELAY2 DEF ON': {'ucl': 1.0, 'lcl': 1.0}, 'RELAY2 DEF OFF': {'ucl': 0.0, 'lcl': 0.0}, 'RELAY3 FAN ON': {'ucl': 1.0, 'lcl': 1.0}, 'RELAY3 FAN OFF': {'ucl': 0.0, 'lcl': 0.0}, 'DISP MOSI HIGH': {'ucl': 5.15, 'lcl': 5.15}, 'DISP MOSI LOW': {'ucl': 0.15, 'lcl': 0.15}, 'DISP MISO HIGH': {'ucl': 5.15, 'lcl': 5.15

In [88]:
df_type1[df_type1['Status']=='0:실패, 1:성공']

date     qty AC 220V 전원 인가 AC 220V 전원 검사 DC12V 전원 검사 DC 5V 전원 검사  \
0  20230322  102431          None         221.4       12.02        5.06   

  TEST 시작  COM0 MODE 485 H/W MODEL F/W VERSION  ... DISP MISO HIGH  \
0     1.0              0      None        None  ...           None   

  DISP MISO LOW DISP CLK HIGH DISP CLK LOW DISP STB HIGH DISP STB LOW  \
0          None          None         None          None         None   

  AC 220V 전원 OFF      date     qty      Status  
0           None  20230322  102431  0:실패, 1:성공  

[1 rows x 34 columns]